First, import needed modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

Initalize variables

In [2]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [3]:
data = pd.read_csv(f'{path}/data/btw_treatment.csv', encoding = 'UTF-8-SIG', sep=',')
#data = data.set_index(['AGS', 'year'])
data.head(10)

,AGS,GEN,year,post_2005,post_2009,post_2013,post_2017,Land,Erststimme,Zweitstimme,...,treated_30*post_2013,treated_30*post_2017,treated_50*post_2005,treated_50*post_2009,treated_50*post_2013,treated_50*post_2017,treated_100*post_2005,treated_100*post_2009,treated_100*post_2013,treated_100*post_2017
0,1001000,Flensburg,2002.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001000,Flensburg,2002.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1001000,Flensburg,2005.0,1.0,0.0,0.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1001000,Flensburg,2005.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1001000,Flensburg,2009.0,1.0,1.0,0.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
5,1001000,Flensburg,2009.0,1.0,1.0,0.0,0.0,1.0,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
6,1001000,Flensburg,2013.0,1.0,1.0,1.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
7,1001000,Flensburg,2013.0,1.0,1.0,1.0,0.0,1.0,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
8,1001000,Flensburg,2017.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
9,1001000,Flensburg,2017.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0


View summary statistics

In [4]:
columns = ['AGS', 'year', 'Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere', 'treated_0', 'treated_50', 'treated_100']
data.describe().to_csv(f'{path}/tables/descriptives.csv', sep=';', encoding='utf-8-sig', decimal=',', columns=columns, float_format='%.2f')
data.describe()

,AGS,year,post_2005,post_2009,post_2013,post_2017,Land,Erststimme,Zweitstimme,Wahlberechtigte,...,treated_30*post_2013,treated_30*post_2017,treated_50*post_2005,treated_50*post_2009,treated_50*post_2013,treated_50*post_2017,treated_100*post_2005,treated_100*post_2009,treated_100*post_2013,treated_100*post_2017
count,1.033820e+05,103382.000000,103382.000000,103382.000000,103382.000000,103382.000000,103382.000000,51691.0,51691.0,1.033820e+05,...,103382.000000,103382.00000,103382.000000,103382.000000,103382.000000,103382.000000,103382.000000,103382.000000,103382.000000,103382.000000
mean,8.032481e+06,2009.514055,0.814281,0.627285,0.428121,0.212397,7.759397,1.0,1.0,5.626048e+03,...,0.096052,0.04761,0.283589,0.214699,0.144687,0.071657,0.484436,0.368381,0.249386,0.123484
std,4.061416e+06,5.373376,0.388882,0.483529,0.494809,0.409006,4.063351,0.0,0.0,3.432014e+04,...,0.294663,0.21294,0.450742,0.410615,0.351786,0.257920,0.499760,0.482368,0.432660,0.328993
min,1.001000e+06,2002.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.0,1.0,0.000000e+00,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.436012e+06,2005.000000,1.000000,0.000000,0.000000,0.000000,6.000000,1.0,1.0,5.250000e+02,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.340203e+06,2009.000000,1.000000,1.000000,0.000000,0.000000,7.000000,1.0,1.0,1.387000e+03,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.672136e+06,2013.000000,1.000000,1.000000,1.000000,0.000000,9.000000,1.0,1.0,4.136000e+03,...,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
max,1.607706e+07,2017.000000,1.000000,1.000000,1.000000,1.000000,16.000000,1.0,1.0,2.505718e+06,...,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Split data in Erst- and Zweitstimme

In [5]:
erst = data[data['Erststimme'] == 1]
zweit = data[data['Zweitstimme'] == 1]
erst.head(10)

,AGS,GEN,year,post_2005,post_2009,post_2013,post_2017,Land,Erststimme,Zweitstimme,...,treated_30*post_2013,treated_30*post_2017,treated_50*post_2005,treated_50*post_2009,treated_50*post_2013,treated_50*post_2017,treated_100*post_2005,treated_100*post_2009,treated_100*post_2013,treated_100*post_2017
0,1001000,Flensburg,2002.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1001000,Flensburg,2005.0,1.0,0.0,0.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1001000,Flensburg,2009.0,1.0,1.0,0.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
6,1001000,Flensburg,2013.0,1.0,1.0,1.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
8,1001000,Flensburg,2017.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
10,1002000,Kiel,2002.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,1002000,Kiel,2005.0,1.0,0.0,0.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,1002000,Kiel,2009.0,1.0,1.0,0.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,1002000,Kiel,2013.0,1.0,1.0,1.0,0.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,1002000,Kiel,2017.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Calculate average vote shares in all years by treated 50

In [6]:
erst_avg = erst[['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere', 'treated_50', 'year']].groupby(['treated_50', 'year'], as_index = False).mean()
zweit_avg = zweit[['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere', 'treated_50', 'year']].groupby(['treated_50', 'year'], as_index = False).mean()
erst_avg.head(10)

,treated_50,year,Union,SPD,FDP,Linke,Grüne,Andere
0,0.0,2002.0,47.774701,36.248840,5.652426,2.891529,4.539332,1.018203
1,0.0,2005.0,46.287267,33.046172,4.947033,6.292648,4.357080,2.925504
2,0.0,2009.0,42.580391,23.099917,10.177012,10.847620,7.445948,3.597763
3,0.0,2013.0,50.113435,23.738684,2.225452,8.790360,5.176069,8.138482
4,0.0,2017.0,40.907351,20.604912,6.366564,7.757606,5.731184,17.144018
5,1.0,2002.0,45.839709,39.168422,6.008737,2.389504,4.168325,0.751704
6,1.0,2005.0,45.724734,35.502430,4.780996,5.359302,3.866982,2.801021
7,1.0,2009.0,43.385968,26.065653,9.838866,8.939487,7.122544,2.620851
8,1.0,2013.0,49.811752,27.603617,2.315732,6.700615,5.198835,6.673383
9,1.0,2017.0,42.717261,23.522352,6.396367,6.297501,5.795939,13.962632


Split data into treated and untreated

In [7]:
erst_avg_treated = erst_avg[erst_avg['treated_50'] == 1]
erst_avg_untreated = erst_avg[erst_avg['treated_50'] == 0]
zweit_avg_treated = zweit_avg[zweit_avg['treated_50'] == 1]
zweit_avg_untreated = zweit_avg[zweit_avg['treated_50'] == 0]
erst_avg_treated.head(10)

,treated_50,year,Union,SPD,FDP,Linke,Grüne,Andere
5,1.0,2002.0,45.839709,39.168422,6.008737,2.389504,4.168325,0.751704
6,1.0,2005.0,45.724734,35.502430,4.780996,5.359302,3.866982,2.801021
7,1.0,2009.0,43.385968,26.065653,9.838866,8.939487,7.122544,2.620851
8,1.0,2013.0,49.811752,27.603617,2.315732,6.700615,5.198835,6.673383
9,1.0,2017.0,42.717261,23.522352,6.396367,6.297501,5.795939,13.962632


Plot the average vote shares over years for treated and untreated municipalities

In [8]:
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 10), dpi=300)
#set x labels
plt.setp(axes, xticks=[2002,2005,2009,2013,2017])
# title for entire figure
fig.suptitle('Average Vote Shares', fontsize=20)

# edit subplots
axes[0, 0].set_title('Union', fontsize=12)
axes[0, 0].set_ylabel('% Votes', fontsize=10)
axes[0, 0].plot(erst_avg_treated['year'], erst_avg_treated['Union'], color='#000000', marker = 'o', label='Treated')
axes[0, 0].plot(erst_avg_untreated['year'], erst_avg_untreated['Union'], color='#7f7f7f', marker = 'o', label='Untreated')
axes[0, 0].legend(loc='lower left')
axes[0, 0].axvline(x=2013, color='red', linestyle='--')

axes[0, 1].set_title('SPD', fontsize=12)
axes[0, 1].set_ylabel('% Votes', fontsize=10)
axes[0, 1].plot(erst_avg_treated['year'], erst_avg_treated['SPD'], color='#ff1100', marker = 'o', label='Treated')
axes[0, 1].plot(erst_avg_untreated['year'], erst_avg_untreated['SPD'], color='#ff7066', marker = 'o', label='Untreated')
axes[0, 1].legend(loc='lower left')
axes[0, 1].axvline(x=2013, color='red', linestyle='--')

axes[1, 0].set_title('FDP', fontsize=12)
axes[1, 0].set_ylabel('% Votes', fontsize=10)
axes[1, 0].plot(erst_avg_treated['year'], erst_avg_treated['FDP'], color='#ffed00', marker = 'o', label='Treated')
axes[1, 0].plot(erst_avg_untreated['year'], erst_avg_untreated['FDP'], color='#fff799', marker = 'o', label='Untreated')
axes[1, 0].legend(loc='upper left')
axes[1, 0].axvline(x=2013, color='red', linestyle='--')

axes[1, 1].set_title('Linke', fontsize=12)
axes[1, 1].set_ylabel('% Votes', fontsize=10)
axes[1, 1].plot(erst_avg_treated['year'], erst_avg_treated['Linke'], color='#cc0055', marker = 'o', label='Treated')
axes[1, 1].plot(erst_avg_untreated['year'], erst_avg_untreated['Linke'], color='#e06699', marker = 'o', label='Untreated')
axes[1, 1].legend(loc='upper left')
axes[1, 1].axvline(x=2013, color='red', linestyle='--')

axes[2, 0].set_title('Grüne', fontsize=12)
axes[2, 0].set_ylabel('% Votes', fontsize=10)
axes[2, 0].plot(erst_avg_treated['year'], erst_avg_treated['Grüne'], color='#3a6e23', marker = 'o', label='Treated')
axes[2, 0].plot(erst_avg_untreated['year'], erst_avg_untreated['Grüne'], color='#62bb3b', marker = 'o', label='Untreated')
axes[2, 0].legend(loc='upper left')
axes[2, 0].axvline(x=2013, color='red', linestyle='--')

axes[2, 1].set_title('Andere', fontsize=12)
axes[2, 1].set_ylabel('% Votes', fontsize=10)
axes[2, 1].plot(erst_avg_treated['year'], erst_avg_treated['Andere'], color='darkgrey', marker = 'o', label='Treated')
axes[2, 1].plot(erst_avg_untreated['year'], erst_avg_untreated['Andere'], color='lightgrey', marker = 'o', label='Untreated')
axes[2, 1].legend(loc='upper left')
axes[2, 1].axvline(x=2013, color='red', linestyle='--')

# Save figure
fig.savefig(f'{path}/figures/vote_shares.png')
plt.close()

Same plot for presentations

In [9]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10), dpi=300)
#set x labels
plt.setp(axes, xticks=[2002,2005,2009,2013,2017])
# title for entire figure
fig.suptitle('Average Vote Shares', fontsize=20)

# edit subplots
axes[0, 0].set_title('Union', fontsize=12)
axes[0, 0].set_ylabel('% Votes', fontsize=10)
axes[0, 0].plot(erst_avg_treated['year'], erst_avg_treated['Union'], color='#000000', marker = 'o', label='Treated')
axes[0, 0].plot(erst_avg_untreated['year'], erst_avg_untreated['Union'], color='#7f7f7f', marker = 'o', label='Untreated')
axes[0, 0].legend(loc='lower left')
axes[0, 0].axvline(x=2013, color='red', linestyle='--')

axes[0, 1].set_title('SPD', fontsize=12)
axes[0, 1].set_ylabel('% Votes', fontsize=10)
axes[0, 1].plot(erst_avg_treated['year'], erst_avg_treated['SPD'], color='#ff1100', marker = 'o', label='Treated')
axes[0, 1].plot(erst_avg_untreated['year'], erst_avg_untreated['SPD'], color='#ff7066', marker = 'o', label='Untreated')
axes[0, 1].legend(loc='lower left')
axes[0, 1].axvline(x=2013, color='red', linestyle='--')

axes[0, 2].set_title('FDP', fontsize=12)
axes[0, 2].set_ylabel('% Votes', fontsize=10)
axes[0, 2].plot(erst_avg_treated['year'], erst_avg_treated['FDP'], color='#ffed00', marker = 'o', label='Treated')
axes[0, 2].plot(erst_avg_untreated['year'], erst_avg_untreated['FDP'], color='#fff799', marker = 'o', label='Untreated')
axes[0, 2].legend(loc='upper left')
axes[0, 2].axvline(x=2013, color='red', linestyle='--')

axes[1, 0].set_title('Linke', fontsize=12)
axes[1, 0].set_ylabel('% Votes', fontsize=10)
axes[1, 0].plot(erst_avg_treated['year'], erst_avg_treated['Linke'], color='#cc0055', marker = 'o', label='Treated')
axes[1, 0].plot(erst_avg_untreated['year'], erst_avg_untreated['Linke'], color='#e06699', marker = 'o', label='Untreated')
axes[1, 0].legend(loc='upper left')
axes[1, 0].axvline(x=2013, color='red', linestyle='--')

axes[1, 1].set_title('Grüne', fontsize=12)
axes[1, 1].set_ylabel('% Votes', fontsize=10)
axes[1, 1].plot(erst_avg_treated['year'], erst_avg_treated['Grüne'], color='#3a6e23', marker = 'o', label='Treated')
axes[1, 1].plot(erst_avg_untreated['year'], erst_avg_untreated['Grüne'], color='#62bb3b', marker = 'o', label='Untreated')
axes[1, 1].legend(loc='upper left')
axes[1, 1].axvline(x=2013, color='red', linestyle='--')

axes[1, 2].set_title('Andere', fontsize=12)
axes[1, 2].set_ylabel('% Votes', fontsize=10)
axes[1, 2].plot(erst_avg_treated['year'], erst_avg_treated['Andere'], color='darkgrey', marker = 'o', label='Treated')
axes[1, 2].plot(erst_avg_untreated['year'], erst_avg_untreated['Andere'], color='lightgrey', marker = 'o', label='Untreated')
axes[1, 2].legend(loc='upper left')
axes[1, 2].axvline(x=2013, color='red', linestyle='--')

# Save figure
fig.savefig(f'{path}/figures/vote_shares_pres.png')
plt.close()